In [1]:
import time, datetime
import copy
import os
import sys
import warnings
import torch
import numpy as np
from loguru import logger
import yaml
from utils import dumb_reward_plot
import gym
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import assistive_gym

import torch


from utils import plot_reward, plot_index
from mpc.mpc_ar import MPC
from baselines.NP_epi import NP

def load_config(config_path="config.yml"):
    if os.path.isfile(config_path):
        f = open(config_path)
        return yaml.load(f, Loader=yaml.FullLoader)
    else:
        raise Exception("Configuration file is not found in the path: "+config_path)

In [2]:
# config = load_config('config/config_cpstable_np.yml')
config = load_config('config/config_ar.yml')
mpc_config = config['mpc_config']
gym_config = config['gym_config']
render = gym_config['render']
np_config = config['NP_config']

model = NP(NP_config=np_config)
logger.info('Using model: {}', model.name)

mpc_controller = MPC(mpc_config=mpc_config)

env = gym.make("FeedingJacoHuman-v0")
log_name = None


2020-08-15 18:00:20.903 | INFO     | __main__:<module>:9 - Using model: NP
Using TensorFlow backend.
/home/baiming/anaconda3/envs/robustnp/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [3]:
"""NP pretrain"""

pretrain_episodes = 1
for task_idx in range(1):
    a1 = np.random.uniform(-0.2, 0.2)
    a2 = np.random.uniform(-0.2, 0.2)
    for epi in range(pretrain_episodes):
        obs = env.reset()
        done = False
        mpc_controller.reset()
        while not done:
            action = env.action_space.sample()
            action[-4] = a1
            action[-1] = a2
            obs_next, reward, done, _ = env.step(action)
            model.data_process([0, obs, action[:7], obs_next - obs])
            obs = obs_next
    model.reset()
    model.train()
# torch.save(model.model.state_dict(), './misc/log/model_test.pth')

In [3]:
# log_name == None


"""testing the model with MPC while training """
test_episode = 1
test_epoch = 1
save_every = 5
log = []
model.reset()
for ep in range(test_epoch):
    task_steps = 0
    a1 = np.random.uniform(-0.2, 0.2)
    a2 = np.random.uniform(-0.2, 0.2)
    human_action = np.array([a1, 0, 0, a2])
    for epi in range(test_episode):
        acc_reward = 0
        obs = env.reset()
        O, A, R, acc_reward, done, V = [], [], [], 0, False, []
        mpc_controller.reset()
        while not done:
            if task_steps > 0:
                robot_action = mpc_controller.act(model=model, state=obs)
            else:
                robot_action = np.zeros(7)
            action = np.concatenate((robot_action, human_action))
            obs_next, reward, done, info = env.step(action)
            task_steps += 1
            A.append(action)
            O.append(obs_next)
            R.append(reward)
            model.data_process([0, obs, action[:7], obs_next - obs])
            obs = obs_next
            acc_reward += reward
    #             print('task: ', task_idx,'step: ', i, 'acc_reward: ', acc_reward, 'violation_rate: ', sum(V)/len(V))
        print(ep, 'acc_reward: ', acc_reward, 'success_food',info['task_success'], 'spilled_food',info['spilled_food'], 'distance', np.linalg.norm(obs[6:9]))
        if ep % save_every == save_every -1:
            torch.save(model.model.state_dict(), './log/ar_np_ju_{}.pth'.format(ep))
            if log_name is None:
                log_name = time.strftime("%Y%m%d_%H%M%S")
            torch.save(model.model.state_dict(), './log/ar_np_ju_{}.pth'.format(ep))
            print('model saved at'+log_name)
#         if log_name is None:
#             log_name = datetime.datetime.now()
#             path = './misc/log/np_adaptation' + log_name.strftime("%d-%H-%M") + '.npy'
#     #                 print(path)
#             np.save(path, log, allow_pickle=True)
#             dumb_reward_plot(path)
    model.reset()
    model.train()

torch.Size([1, 1, 30]) torch.Size([1, 200, 30])
R torch.Size([1, 200, 2])
Z torch.Size([1, 200, 2])
torch.Size([1, 1, 30]) torch.Size([1, 200, 30])
R torch.Size([1, 200, 2])
Z torch.Size([1, 200, 2])
torch.Size([1, 1, 30]) torch.Size([1, 200, 30])
R torch.Size([1, 200, 2])
Z torch.Size([1, 200, 2])
torch.Size([1, 1, 30]) torch.Size([1, 200, 30])
R torch.Size([1, 200, 2])
Z torch.Size([1, 200, 2])
torch.Size([1, 1, 30]) torch.Size([1, 200, 30])
R torch.Size([1, 200, 2])
Z torch.Size([1, 200, 2])
torch.Size([1, 1, 30]) torch.Size([1, 200, 30])
R torch.Size([1, 200, 2])
Z torch.Size([1, 200, 2])
torch.Size([1, 1, 30]) torch.Size([1, 200, 30])
R torch.Size([1, 200, 2])
Z torch.Size([1, 200, 2])
torch.Size([1, 1, 30]) torch.Size([1, 200, 30])
R torch.Size([1, 200, 2])
Z torch.Size([1, 200, 2])
torch.Size([1, 1, 30]) torch.Size([1, 200, 30])
R torch.Size([1, 200, 2])
Z torch.Size([1, 200, 2])
torch.Size([1, 1, 30]) torch.Size([1, 200, 30])
R torch.Size([1, 200, 2])
Z torch.Size([1, 200, 2])


KeyboardInterrupt: 

In [ ]:
# torch.save(model.model.state_dict(), './misc/log/robust_model_latent-' + log_name.strftime("%d-%H-%M") + '.pth')

In [ ]:
# print(path)